In [ ]:
! pip install -q kaggle

In [ ]:
# upload kaggle json
 ! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d antoinecarpentier/redditrplacecsv

100% 11.6G/11.6G [01:22<00:00, 201MB/s]
100% 11.6G/11.6G [01:22<00:00, 151MB/s]


In [ ]:
! unzip redditrplacecsv.zip



Archive:  redditrplacecsv.zip
  inflating: 2022_place_canvas_history.csv  


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
from datetime import datetime

# The length of time in milliseconds after 1970-01-01T00:00:00.000 UTC that
# the first pixel was placed in r/Place 2022.
START_TIME = 1648806250315


def parse_timestamp(timestamp):
    """Convert a YYYY-MM-DD HH:MM:SS.SSS timestamp to milliseconds after the start of r/Place 2022."""
    date_format = "%Y-%m-%d %H:%M:%S.%f"
    try:
        # Remove the UTC timezone from the timestamp and convert it to a POSIX timestamp.
        timestamp = datetime.strptime(timestamp[:-4], date_format).timestamp()
    except ValueError:
        # The timestamp is exactly on the second, so there is no decimal (%f).
        # This happens 1/1000 of the time.
        timestamp = datetime.strptime(timestamp[:-4], date_format[:-3]).timestamp()

    # Convert from a float in seconds to an int in milliseconds
    timestamp *= 1000.0
    timestamp = int(timestamp)

    # The earliest timestamp is 1648806250315, so subtract that from each timestamp
    # to get the time in milliseconds since the beginning of the experiment.
    timestamp -= START_TIME

    return timestamp


# Parse a sample timestamp.
parse_timestamp("2022-04-04 00:55:40.375 UTC")


227490060

In [ ]:
def parse_pixel_color(pixel_color):
    """Convert a hex color code to an integer key."""
    hex_to_key = {
        "#000000": 0,
        "#00756F": 1,
        "#009EAA": 2,
        "#00A368": 3,
        "#00CC78": 4,
        "#00CCC0": 5,
        "#2450A4": 6,
        "#3690EA": 7,
        "#493AC1": 8,
        "#515252": 9,
        "#51E9F4": 10,
        "#6A5CFF": 11,
        "#6D001A": 12,
        "#6D482F": 13,
        "#7EED56": 14,
        "#811E9F": 15,
        "#898D90": 16,
        "#94B3FF": 17,
        "#9C6926": 18,
        "#B44AC0": 19,
        "#BE0039": 20,
        "#D4D7D9": 21,
        "#DE107F": 22,
        "#E4ABFF": 23,
        "#FF3881": 24,
        "#FF4500": 25,
        "#FF99AA": 26,
        "#FFA800": 27,
        "#FFB470": 28,
        "#FFD635": 29,
        "#FFF8B8": 30,
        "#FFFFFF": 31,
    }

    return hex_to_key[pixel_color]

In [ ]:
def split_coords_single_points(points):
    """
    Given a dataframe containing only rows that have single-point
    coordinates, split the coordinates into x and y columns.
    """

    # Convert the coordinate column to a list of strings.
    points["coordinate"] = points["coordinate"].apply(lambda x: x.split(","))

    # Create new x and y columns from the coordinate column.
    points["x"] = points["coordinate"].apply(lambda x: x[0]).astype("uint16")
    points["y"] = points["coordinate"].apply(lambda x: x[1]).astype("uint16")

    # Drop the coordinate column.
    del points["coordinate"]

    return points

In [ ]:
def process_chunk_v2(chunk, df,counter,mapping):
    chunk["timestamp"] = chunk["timestamp"].astype("uint32")
    chunk["pixel_color"] = chunk["pixel_color"].astype("uint8")
    
    # per rimuovere le azioni degli admin
    chunk.drop(chunk[chunk["coordinate"].str.count(",") == 3].index,inplace=True)

    chunk = split_coords_single_points(chunk)
    
    for user in chunk.user_id:
        if not user in mapping:
            mapping[user] = counter
            counter += 1

    chunk["user_id"] = chunk["user_id"].map(mapping)
    chunk["user_id"] = chunk["user_id"].astype("uint32")

    df = pd.concat((df, chunk), ignore_index=True)

    return df,counter,mapping

In [ ]:
CHUNK_SIZE = 1000000
#CHUNK_SIZE = 1_000


def trim(infile_path, outfile_path):
    """Trim the infile data and write it to outfile."""
    df = pd.DataFrame(columns=["timestamp", "user_id","pixel_color", "x", "y"])
    df["timestamp"] = df["timestamp"].astype("uint32")
    df["pixel_color"] = df["pixel_color"].astype("uint8")
    df["user_id"] = df["user_id"].astype("uint32")

    df["x"] = df["x"].astype("uint16")
    df["y"] = df["y"].astype("uint16")

    mapping = {}
    counter = 0df.head()
    
    with pd.read_csv(
        infile_path,
        usecols=["timestamp", "user_id","pixel_color", "coordinate"],
        converters={
            "timestamp": parse_timestamp,
            "pixel_color": parse_pixel_color,
        },
        chunksize=CHUNK_SIZE,
        engine="c",
        #compression="gzip",
    ) as csv:
        for chunk in tqdm(csv):
            df, counter, mapping = process_chunk_v2(chunk, df,counter,mapping)
            #print("counter é : ", counter)
            #print("len unique(): ", len(df.user_id.unique()))
            #print("len unique(): ", len(df.user_id.unique()))


    #df.sort_values("timestamp", inplace=True, ignore_index=True)    
    df.to_csv('my-file.csv', sep=',', index=False)
    print("final counter",counter)
    return df

SyntaxError: ignored

161 iterazioni  1h37'11", 36.22s/it

final counter 10381144



In [ ]:
l = list(range(0,100))

l[::5]

[1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81, 86, 91, 96]

In [ ]:
infile_path = "2022_place_canvas_history.csv"
outfile_path = "reddit_trimmed.csv" 

df_trim = trim(infile_path, outfile_path)
df_trim

161it [1:37:11, 36.22s/it]


final counter 10381144


,timestamp,user_id,pixel_color,x,y
0,227381262,0,5,826,1048
1,227383443,1,17,583,1031
2,227384370,2,11,1873,558
3,227447226,3,2,1627,255
4,227465992,4,17,49,1478
...,...,...,...,...,...
160353080,311389751,2119667,31,408,493
160353081,311389830,862794,31,1232,312
160353082,311389857,2778740,31,770,866
160353083,311389880,1591560,31,1046,1721


In [ ]:
df_trim.sort_values("timestamp", inplace=True, ignore_index=True)
df_trim

,timestamp,user_id,pixel_color,x,y
0,10800000,4068929,14,42,42
1,10812356,4068930,3,999,999
2,10816311,4068931,7,44,42
3,10821388,4068932,21,2,2
4,10834094,4068933,7,23,23
...,...,...,...,...,...
160353080,311389751,2119667,31,408,493
160353081,311389830,862794,31,1232,312
160353082,311389857,2778740,31,770,866
160353083,311389880,1591560,31,1046,1721


In [ ]:
df_trim.to_csv("reddit_trimmed.csv.zip", 
           index=False, 
           compression="zip")

In [ ]:
#min(df_trim.timestamp) #10_800_000

10800000

In [ ]:
1_648_924_835 - 1_648_806_250_315

-1647157325480

In [ ]:
datetime.fromtimestamp(1_648_806_250_315) 

ValueError: ignored

In [ ]:
datetime.fromtimestamp(1_648_924_835) 

datetime.datetime(2022, 4, 2, 18, 40, 35)

In [ ]:
# questo sono gli among us di un preciso timestamp
timestamp = 1_648_924_835 #nel dataset delle immagini

x = [  6,   6,  41,  43,  43,  44,  45,  49,  51,  51,  51,  52,  63,
        63,  68,  75,  79,  79,  80,  86,  87,  89,  89,  89,  90,  91,
        94,  95,  96, 104, 106, 124, 125, 126, 126, 126, 126, 126, 128,
        140, 140, 148, 149, 149, 151, 155, 156, 182, 185, 189, 189, 192,
        201, 206, 221, 230, 236, 243, 248, 248, 248, 248, 249, 249, 258,
        270, 275, 276, 283, 289, 296, 303, 311, 335, 341, 342, 344, 347,
        357, 360, 361, 364, 366, 369, 369, 369, 370, 374, 375, 378, 392,
        393, 397, 418, 423, 426, 427, 436, 438, 440, 451, 459, 467, 468,
        469, 472, 473, 475, 477, 477, 479, 482, 482, 488, 488, 490, 490,
        492, 492, 505, 508, 512, 512, 523, 533, 535, 542, 544, 544, 545,
        545, 546, 546, 546, 549, 551, 559, 559, 566, 566, 570, 570, 576,
        577, 603, 603, 604, 605, 608, 609, 611, 624, 636, 642, 665, 670,
        672, 679, 696, 696, 697, 697, 697, 697, 697, 697, 697, 703, 710,
        714, 715, 718, 718, 719, 722, 728, 732, 732, 732, 734, 734, 735,
        737, 737, 737, 738, 742, 744, 745, 747, 749, 750, 750, 750, 753,
        754, 754, 761, 762, 762, 765, 767, 768, 772, 773, 773, 774, 774,
        775, 777, 780, 780, 780, 780, 780, 780, 780, 781, 782, 784, 787,
        789, 789, 790, 791, 791, 791, 791, 791, 791, 792, 794, 798, 802,
        805, 808, 812, 817, 821, 822, 827, 833, 835, 838, 853, 857, 863,
        865, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881,
        881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881,
        881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881, 881,
        881, 885, 885, 885, 886, 886, 886, 886, 886, 886, 886, 886, 886,
        886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886,
        886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886, 886,
        886, 886, 886, 886, 886, 887, 891, 891, 891, 891, 891, 891, 891,
        891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891,
        891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891, 891,
        891, 891, 891, 891, 892, 893, 895, 895, 895, 895, 895, 895, 895,
        895, 895, 895, 895, 895, 895, 895, 896, 896, 896, 896, 896, 896,
        896, 896, 896, 896, 896, 896, 896, 896, 896, 896, 896, 896, 896,
        901, 901, 908, 917, 922, 928, 931, 931, 934, 936, 938, 941, 943,
        943, 943, 943, 943, 948, 948, 948, 948, 948, 953, 953, 954, 962,
        965, 966, 975, 981, 981, 982, 982, 987, 991, 991, 995, 996]

y = [ 449,  612,  311,  316,  321, 1660,  278,  710,  739, 1573, 1577,
        1383,  404, 1948,  249,  306,  254,  579,  148,  408, 1115,  595,
        777,  782, 1112, 1202, 1545,  454, 1269, 1180, 1820, 1433, 1969,
        1360, 1364, 1368, 1372, 1381, 1030,  315,  319,  899, 1214, 1509,
        941,  583,  618, 1604,  546,  942, 1168, 1701, 1146, 1193,  505,
        1794,  827,  921,  182,  192,  223, 1466,   18,  393,  140,  452,
        1977, 1544, 1480, 1949,  908, 1575, 1864, 1129,   77,   94, 1804,
        1597,  122, 1942,  127,  124, 1648,  594,  612, 1624,  555, 1169,
        1690, 1700, 1398, 1237, 1603, 1794, 1679,  584, 1223,  266, 1939,
        1550, 1929, 1757, 1870,   46,    6,  943,   47,    2,  201,  219,
          5,   46,  156,    1, 1551,  142, 1236,  284, 1490, 1159, 1486,
        1557, 1704, 1480, 1843,  198, 1253, 1191, 1207, 1748, 1802, 1736,
        1740, 1752, 1549,  654, 1831, 1834, 1607, 1829, 1862, 1866,  930,
        1171,  370, 1288,  335,  353,  338, 1718, 1421, 1822,  339, 1370,
        1862, 1789, 1723, 1834,  586, 1700, 1324, 1707, 1820, 1828, 1845,
        1849, 1861, 1828,  616,  593,  640,  613,  629,  593, 1776,  629,
        921, 1016, 1776,  620, 1017,  920, 1533, 1540, 1776,  613, 1776,
        1616, 1617, 1051,  622,  646,  995, 1099,  642,  633, 1801,  770,
        576,  581,  948,  582,  577,  663,  623,  627,  637,  642,  578,
        1087,  649, 1063, 1066, 1076, 1087, 1090, 1095,  641, 1380,  593,
        620,  612, 1680,  607,  602, 1007, 1016, 1057, 1100, 1120,  927,
        593,  585, 1387, 1393,  460,  845,  289,  281, 1891, 1606,  511,
        1139, 1168, 1185,  516,  526,  229, 1621, 1625, 1629, 1637, 1641,
        1645, 1649, 1653, 1657, 1661, 1665, 1669, 1673, 1677, 1681, 1685,
        1689, 1693, 1697, 1701, 1705, 1709, 1713, 1721, 1729, 1733, 1737,
        1741, 1745, 1749, 1753, 1757, 1761, 1765, 1773, 1777, 1781, 1785,
        1789,  437, 1785, 1793, 1625, 1629, 1633, 1637, 1641, 1645, 1649,
        1653, 1657, 1661, 1665, 1669, 1673, 1677, 1681, 1685, 1689, 1693,
        1697, 1701, 1705, 1709, 1713, 1717, 1721, 1725, 1729, 1733, 1737,
        1741, 1745, 1749, 1757, 1761, 1765, 1769, 1773, 1777, 1781, 1789,
        1816, 1610, 1614, 1621, 1625, 1629, 1633, 1637, 1641, 1649, 1653,
        1657, 1661, 1665, 1669, 1673, 1677, 1681, 1689, 1693, 1697, 1701,
        1705, 1709, 1713, 1717, 1721, 1725, 1729, 1733, 1741, 1745, 1749,
        1753, 1757, 1761, 1765, 1769, 1503,  436, 1610, 1614, 1621, 1625,
        1629, 1633, 1637, 1641, 1645, 1649, 1653, 1657, 1661, 1673,  613,
        1669, 1677, 1693, 1697, 1701, 1705, 1713, 1717, 1729, 1733, 1737,
        1741, 1745, 1749, 1753, 1757, 1761, 1765, 1669, 1673, 1638,  463,
        108, 1847, 1410, 1418, 1474, 1493, 1581, 1605,   70,   74,   78,
          82,   86,   70,   74,   78,   82,   86,   74,   78, 1071, 1317,
          29,   22, 1675,   89, 1836,   59, 1841, 1854,   81,   85, 1567,
          89]

len(x), len(y)

(441, 441)

In [ ]:
for i,j in zip(x,y):
  print(i,j)
  

6 449
